In [ ]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA256


def fnv_1a(msg: bytes, fnv_offset: int, fnv_prime: int, fnv_size: int) -> int:
    h = fnv_offset
    for i in range(0, len(msg)):
        h ^^= msg[i]
        h = (h * fnv_prime) % fnv_size
    return h


def calc_ord_element(fnv_prime: int, fnv_size: int) -> int:
    pw = 2
    for _ in range(int(fnv_size).bit_length()):
        if pow(fnv_prime, pw, fnv_size) == 1:
            return pw
        pw *= 2


def calc_discrete_log_for_offset(fnv_offset: int, fnv_prime: int, fnv_size: int) -> (int, int):
    Z = Mod(fnv_prime, fnv_size)
    num_in_fnv_prime_group = fnv_offset
    while True:
        try:
            dl = discrete_log(num_in_fnv_prime_group, Z)
        except:
            num_in_fnv_prime_group += 2
        else:
            break
    return dl, num_in_fnv_prime_group


def create_zero_hash_msg(fnv_offset: int, fnv_prime: int, fnv_size: int) -> bytes:
    ord_fnv_prime = calc_ord_element(fnv_prime, fnv_size)
    dlog, new_offset = calc_discrete_log_for_offset(fnv_offset, fnv_prime, fnv_size)
    msg = bytearray()
    msg.append(fnv_offset ^^ new_offset)
    [msg.append(0x00) for _ in range(ord_fnv_prime - dlog - 1)]
    msg.append(0x01)
    return bytes(msg)


# Proof of Concept

# 1. Correctness FNV-1a implementation for 32 bits hash length

fnv_32_offset = 0x811c9dc5
fnv_32_prime = 0x01000193
fnv_32_size = 2**32

assert fnv_1a(b'hello world', fnv_32_offset, fnv_32_prime, fnv_32_size) == 3582672807

# 2. Correctness of creating a message with a zero hash

fnv_16_offset = 28659
fnv_16_prime = 43261
fnv_16_size = 2**16

message = create_zero_hash_msg(fnv_16_offset, fnv_16_prime, fnv_16_size)
assert fnv_1a(message, fnv_16_offset, fnv_16_prime, fnv_16_size) == 0

# 3. Correctness of the secret calculation for the task

ord_prime = calc_ord_element(fnv_16_prime, fnv_16_size)
x = calc_discrete_log_for_offset(fnv_16_offset, fnv_16_prime, fnv_16_size)[0]
message_len = ord_prime - x + 1
assert message_len == len(message)


# Create task

with open('../task/data.txt', 'r') as f:
    fnv_1024_offset = int(f.readline(), 16)
    fnv_1024_prime = int(f.readline(), 16)
    fnv_1024_size = int(f.readline(), 16)

ord_prime = calc_ord_element(fnv_1024_prime, fnv_1024_size)
x, offset = calc_discrete_log_for_offset(fnv_1024_offset, fnv_1024_prime, fnv_1024_size)

# print(1024 - int(log(fnv_1024_size // ord_prime, 2).n()))
# print(fnv_1024_offset ^^ offset)

# zero_msg = bytearray()
# zero_msg.append(fnv_1024_offset ^^ offset)
# [zero_msg.append(0x00) for _ in range(ord_prime - x - 1)]
# zero_msg.append(0x01)
# assert fnv_1a(bytes(zero_msg), fnv_1024_offset, fnv_1024_prime, fnv_1024_size) == 0
# secret = len(zero_msg)

secret = ord_prime - x + 1

key = SHA256.new(str(secret).encode()).digest()
aes = AES.new(key, AES.MODE_CTR)

with open('flag.txt', 'r') as f:
    flag = bytes(f.readline(), 'ascii')

with open('../task/task.txt', 'w') as f:
    f.write(aes.encrypt(flag).hex() + '\n')
    f.write(aes.nonce.hex() + '\n')


# Solve task

with open('../task/task.txt', 'r') as f:
    ct = bytes.fromhex(f.readline())
    iv = bytes.fromhex(f.readline())

with open('../task/data.txt', 'r') as f:
    fnv_1024_offset = int(f.readline(), 16)
    fnv_1024_prime = int(f.readline(), 16)
    fnv_1024_size = int(f.readline(), 16)

x = calc_discrete_log_for_offset(fnv_1024_offset, fnv_1024_prime, fnv_1024_size)[0]
ord_prime = calc_ord_element(fnv_1024_prime, fnv_1024_size)
key = SHA256.new(str(ord_prime - x + 1).encode()).digest()
aes = AES.new(key, AES.MODE_CTR, nonce=iv)
print(aes.decrypt(ct))